In [1]:
from langchain.agents import initialize_agent, AgentType, Tool
from langchain.memory import ConversationBufferMemory
from langgraph.graph import StateGraph
from langchain.chat_models import ChatOpenAI
from langchain_gigachat.chat_models import GigaChat
from langchain.schema import HumanMessage, SystemMessage, Document, AIMessage
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.tools import DuckDuckGoSearchRun
from langgraph.types import Command
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, END, MessagesState, START
from typing import TypedDict, Optional, Literal, List, Dict
from typing_extensions import TypedDict

import datetime
import warnings
import os
import re

In [2]:
warnings.filterwarnings("ignore")

In [3]:
os.environ["GIGACHAT_CREDENTIALS"] = "Yjc1YWZhNTItMzYwYS00NmU4LTk4YjctZjU4YzAwMDIyMGJmOjJhNWE3YmVhLTAyYzctNGJhNy05NWE3LWEzY2YwNGQzYzZiNw=="

In [ ]:
def save_result(result):
    format_ = "%Y_%m_%dT%H_%M_%S"
    datetime_mark = datetime.datetime.now().strftime(format_)
    
    with open(f"tmp/desc_{datetime_mark}.md", "w") as f:
        f.write(result["description"])

    with open(f"tmp/graph_{datetime_mark}.md", "w") as f:
        f.write(result["graph"])

In [4]:
llm = GigaChat(
    model="GigaChat-2-Max",
    verify_ssl_certs=False,
    profanity_check=False,
    streaming=False,
    max_tokens=8192,
    temperature=0.3,
    repetition_penalty=1.01
)

In [27]:
class AgentState(TypedDict):
    task: str
    messages: List
    description: str
    graph: str
    schema: str
    
class ValidatorResponse(TypedDict):
    """В поле instructions нужно указать рекомендации. В поле status указать статус задачи [FAIL, SUCCESS]"""
    instructions: str
    status: Literal["FAIL", "SUCCESS"]

In [28]:
instruments = """RAG - поиск аналогичных решений по своей базе данных для более корректной генерации;
Web Search - поиск в интернете, если требуется найти актуальную информацию;
API - взаимодействие со сторонними сервисами через API."""

In [29]:
describer_prompt = """Ты - превосходный аналитик. Тебе дается задача {task}. Тебе нужно придумать и описать, как реализовать эту задачу с помощью llm агентов.
Подумай над следующими вопросами: 
- Как придуманная система будет взаимодействовать со сторонними системами (из какой системы будет приходить запрос, куда будет отправляться результат);
- Сколько агентов необходимо для реализации системы: один или несколько; 
- Нужен ли агент оркестратор или можно обойтись без него (Агент оркестратор может быть необходим, если требуется координировать работу нескольких агентов, которые работают вместе);
- Нужен ли агент валидатор или можно обойтись без него (Агент валидатор может быть необходим, если нужно провалидировать работу другого агента и, если работа не соответствует критериям, вернуть задачу на доработку); 
- Какую задачу решает каждый агент, какие инструменты из перечня инструментов {instruments} ему необходимы для решения его задачи;
- С какими системами будет взаимодействовать каждый агент;


Опиши придуманный тобой процесс.
"""

In [81]:
description_validator_prompt = """Ты - Валидатор. Ты прекрасно умеешь критиковать написанные сценарии для достижения наилучшего результата.
Ты работаешь вместе с Аналитиком. Проанализуруй сценарий, который предложил аналитик к задаче {task}. 
Сценарий не должен быть очень подробным: не должны быть описаны контракты между сервисами, структура БД, метрики качества. Должна быть отражена только основная суть. 
Проверь, что сценарий, который предложил аналитик подходит для решения задачи. 
Проверь, что описаны как основные сценарии, так и альтернативные.
Если сценарий нужно улучшать, напиши свои замечания и верни сценарий на дорабоку, ответив FAIL В конце инструкции попроси аналитика заново полностью описать сценарий. 
Если Аналитик хорошо выполнил свою работу и замечаний нет, ответь SUCCESS.

Ответ дай в следующем формате json:
```json
{format}
```
"""

In [82]:
format_ ="""{
    "instructions": "Здесь укажи свои рекомендации или оставь строку пустой, если рекомендаций нет",
    "status": "FAIL" если требуется доработка иначе "SUCCESS"
}"""

In [83]:
graph_maker_prompt = """Ты - превосходный аналитик. Тебе дается задача {task} и описание ее решения {description}.
Выдели основные объекты системы и опиши граф связей между объектами. Если в процессе упомянуты инструменты, изобрази их. Граф должен включать как основной так и альтернативные сценарии.
Граф должент быть простой и понятный, изображать только основные этапы взаимодействия без чрезмерных подробностей.
Для наглядности изобрази схему в формате markdown"""

In [84]:
task_3 = """Требуется создать систему автозаполнения заявок инженеров сопровождения. Как должна работать система:
Есть база данных "ПКАП", с которой можно взаимодействовать через сторонний сервис OpenShift. Сервис Openshift извлекает данные из БД и отдает их системе.
Данные представляют собой заявку по инциденту, которые необходимо заполнить. Система должна проанализировать заявку, выделить причины инциденда и способ его закрытия.
Система должна сравнить проанализированные данные с аналогичными примерами, которые у нее есть. В случае, если найден похожий пример, заполнить заявку основываясь на примере и результаете своего анализа."""

In [90]:
process_describe_agent = create_react_agent(llm, tools=[], checkpointer=MemorySaver())
graph_maker_agent = create_react_agent(llm, tools=[])

In [99]:
def describe_agent_node(state:AgentState)->Command[Literal["describtion_validator"]]:    
    print("Status: describer")
    if "messages" in state and state["messages"]:
        old_messages = state["messages"]
        request = {
            "messages": HumanMessage(content=state["messages"][-1].content)
        }
    else:
        old_messages = []
        request = {
            "messages": describer_prompt.format(task=state["task"], instruments=instruments)
        }
    response = process_describe_agent.invoke(request, config=config)
    if isinstance(response, dict):
        result = response["messages"][-1].content
    else:
        result = response
    return Command(
        update={
            "description": result,
            "messages": old_messages + [HumanMessage(content=result, name="Аналитик")]
        },
        goto="describtion_validator"
    )

def describtion_validator_node(state:AgentState)->Command[Literal["describer", "graph_maker_agent"]]:
    print(f"Status: describtion_validator_node")
    return_node = "describer"
    if not "messages" in state or not state["messages"]:
        return Command(goto=return_node)
    
    prompt = description_validator_prompt.format(task=state["task"], format=format_)
    system = SystemMessage(content=prompt)
    request = [system] + [state["messages"][-1]]

    validator_msgs_fount = len(list(filter(lambda x: x.name=="Валидатор", state["messages"])))
    
    response = llm.invoke(request)
    response = response.content
    print(f"RESPONSE: {response}")
    match_json = re.search(r'```json\s*(.*?)\s*```', response, re.DOTALL)
    if match_json:
        response = eval(match_json.group(1))
    else:
        response = {"status":"SUCCESS", "instructions":""}
    result = response["instructions"]
    print(f"STATUS: {response['status']}\n\nVALIDATOR: {result}")
    
    if response["status"]=="SUCCESS" or validator_msgs_fount+1 > 3:
        goto = "graph_maker_agent"
        return Command(goto=goto)
    else:
        goto = return_node
        return Command(
            update={
            "messages": state["messages"] + [HumanMessage(content=result, name="Валидатор")],
            },
            goto=goto
        )
    
def graph_maker_agent_node(state:AgentState)->Command[Literal[END]]:
    request = graph_maker_prompt.format(task=state["task"], description=state["description"])
    request = {
        "messages": HumanMessage(content=request)
    }
    response = graph_maker_agent.invoke(request)
    if isinstance(response, dict):
        result = response["messages"][-1].content
    else:
        result = response
    return Command(
        update={
            "graph": result
        },
        goto=END
    )

In [100]:
config = {"configurable": {"thread_id": "thread_id123"}}

In [101]:
state_schema = AgentState
builder = StateGraph(state_schema=state_schema)
builder.set_entry_point("describer")
builder.add_node("describer", describe_agent_node)
builder.add_node("describtion_validator", describtion_validator_node)
builder.add_node("graph_maker_agent", graph_maker_agent_node)
graph = builder.compile()

In [102]:
initial_state_2 = {
    "task": task_3,
}

In [127]:
save_result(result_2)

In [103]:
result_2 = graph.invoke(initial_state_2)

Status: describer
Status: describtion_validator_node
RESPONSE: ```json
{
    "instructions": "Сценарий требует доработки. Необходимо уточнить, как именно происходит взаимодействие между агентами и какие действия предпринимаются в случае, если не найдено аналогичного примера. Также стоит добавить описание альтернативных сценариев, например, что происходит, если данные из OpenShift некорректны или база данных недоступна.",
    "status": "FAIL"
}
```
STATUS: FAIL

VALIDATOR: Сценарий требует доработки. Необходимо уточнить, как именно происходит взаимодействие между агентами и какие действия предпринимаются в случае, если не найдено аналогичного примера. Также стоит добавить описание альтернативных сценариев, например, что происходит, если данные из OpenShift некорректны или база данных недоступна.
Status: describer
Status: describtion_validator_node
RESPONSE: ```json
{
    "instructions": "",
    "status": "SUCCESS"
}
```
STATUS: SUCCESS

VALIDATOR: 


In [31]:
# from IPython.display import Image, display

# try:
#     display(Image(graph.get_graph().draw_mermaid_png()))
# except Exception:
#     # You can put your exception handling code here
#     pass

In [17]:
initial_state = {
    "task": task_3,
}

In [5]:
from langgraph.prebuilt import create_react_agent
from langchain_core.prompts import ChatPromptTemplate

In [6]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Разбей задачу на подзадачи и пошагово решай ее. \
                    Если для решения поставленного вопроса нужна информация, задай уточняющий \
                    вопрос в формате [Для решения вопроса мне не хватает информации: информация]"),
        ("placeholder", "{messages}"),
    ]
)

In [8]:
describer_agent_gigachat = create_react_agent(model=llm, tools=[], prompt=prompt)

In [74]:
graph_maker_agent_gigachat = create_react_agent(model=llm, tools=[], prompt=prompt)

In [18]:
messages = {
    "messages": [
        HumanMessage(content=describer_prompt.format(task=initial_state["task"], instruments=instruments))
    ]
}

In [19]:
describer_result = describer_agent_gigachat.invoke(messages)

In [24]:
print(describer_result["messages"][1].content)

### Анализ задачи и постановка подзадач

Для реализации системы автозаполнения заявок инженеров сопровождения необходимо решить следующие подзадачи:

1. **Получение данных из сторонней системы**:
   - Извлечение заявки по инциденту из базы данных "ПКАП" через сервис OpenShift.

2. **Анализ заявки**:
   - Анализ заявки для выделения причин инцидента и способа его закрытия.

3. **Поиск аналогичных примеров**:
   - Поиск аналогичных примеров в базе данных для сравнения с текущей заявкой.

4. **Заполнение заявки**:
   - Заполнение заявки на основе анализа и найденных аналогичных примеров.

5. **Валидация результата**:
   - Проверка корректности заполнения заявки.

6. **Отправка результата**:
   - Отправка заполненной заявки обратно в систему.

### Определение агентов и их задач

Для реализации системы потребуется несколько агентов:

1. **Агент извлечения данных (Data Extraction Agent)**:
   - Задача: Извлечение заявки из базы данных "ПКАП" через сервис OpenShift.
   - Инструменты: API.
   

In [75]:
messages_2 = {
    "messages": [
        HumanMessage(content=describer_result["messages"][1].content)
    ]
}

In [76]:
graph_maker_result = graph_maker_agent_gigachat.invoke(messages_2)

In [81]:
with open("giga_graph.txt", "w") as f:
    f.write(graph_maker_result["messages"][1].content)

In [82]:
with open("giga_desc.txt", "w") as f:
    f.write(describer_result["messages"][1].content)

In [80]:
graph_maker_result["messages"][1].content

'### Шаги для реализации системы автозаполнения заявок инженеров сопровождения:\n\n#### 1. Анализ задачи и постановка целей\n- Определить требования к системе автозаполнения заявок.\n- Определить источники данных (база данных "ПКАП", сервис OpenShift).\n- Определить формат взаимодействия с инженерами сопровождения.\n\n#### 2. Взаимодействие со сторонними системами\n- Реализовать интерфейс для приема заявок от инженеров сопровождения.\n- Реализовать интерфейс для отправки заполненных заявок инженерам сопровождения.\n\n#### 3. Необходимость агентов\n- Разработать агента-анализатора для анализа заявок.\n- Разработать агента-поисковика для поиска аналогичных примеров.\n- Разработать агента-заполнителя для заполнения заявок.\n- Разработать агента-оркестратора для координации работы агентов.\n- Разработать агента-валидатора для проверки корректности заполнения заявок.\n\n#### 4. Необходимость агента-оркестратора\n- Определить задачи и инструменты агента-оркестратора.\n- Реализовать координац

In [73]:
print(describer_result["messages"][1].content)

### 1. Анализ задачи и постановка целей
Система автозаполнения заявок инженеров сопровождения должна:
- Получать данные заявки по инциденту из базы данных "ПКАП" через сервис OpenShift.
- Анализировать заявку, выделяя причины инцидента и способ его закрытия.
- Сравнивать проанализированные данные с аналогичными примерами, которые у нее есть.
- В случае нахождения похожего примера, заполнять заявку на основе примера и результата анализа.

### 2. Взаимодействие со сторонними системами
- **Запрос**: Запрос на анализ и автозаполнение заявки приходит от инженера сопровождения через интерфейс системы.
- **Результат**: Заполненная заявка отправляется обратно инженеру сопровождения через интерфейс системы.

### 3. Необходимость агентов
Для реализации системы потребуется несколько агентов:
- **Агент-анализатор**: анализирует заявку, выделяет причины инцидента и способ его закрытия.
- **Агент-поисковик**: ищет аналогичные примеры в базе данных.
- **Агент-заполнитель**: заполняет заявку на основе